# NBA Database Scraper (2000-Present)

This notebook scrapes Basketball Reference to build a SQLite database of NBA data.

**Tables:**
- `players` — player biographical info
- `teams` — team identifiers and conference/division
- `player_season_stats` — per-game and advanced stats by season
- `team_season_stats` — team record and ratings by season
- `games` — individual game results

**Seasons:** 1999-2000 through 2024-25 (referenced as 2000-2025 in Basketball Reference URLs)

## 1. Imports and Configuration

In [3]:
import requests
from bs4 import BeautifulSoup, Comment
import sqlite3
import time
import re

DB_PATH = "nba.db"
BASE_URL = "https://www.basketball-reference.com"
DELAY = 3

# Season range
SEASONS = list(range(2000, 2026))

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

print(f"Config loaded")

Config loaded


## 2. Database Schema

In [56]:
# Create database tables
# Check if they exist first
def create_tables():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS players (
            player_id TEXT PRIMARY KEY,
            full_name TEXT,
            birth_date TEXT,
            height TEXT,
            weight INTEGER,
            position TEXT,
            shoots TEXT,
            draft_year INTEGER,
            draft_round INTEGER,
            draft_pick INTEGER,
            draft_team TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS teams (
            team_id TEXT PRIMARY KEY,
            team_name TEXT,
            conference TEXT,
            division TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS player_season_stats (
            player_id TEXT,
            season INTEGER,
            team_id TEXT,
            games INTEGER,
            games_started INTEGER,
            minutes REAL,
            points REAL,
            rebounds REAL,
            assists REAL,
            per REAL,
            ts_pct REAL,
            ws REAL,
            bpm REAL,
            vorp REAL,
            PRIMARY KEY (player_id, season, team_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS team_season_stats (
            team_id TEXT,
            season INTEGER,
            wins INTEGER,
            losses INTEGER,
            win_pct REAL,
            pace REAL,
            offensive_rating REAL,
            defensive_rating REAL,
            srs REAL,
            PRIMARY KEY (team_id, season)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS games (
            game_id TEXT PRIMARY KEY,
            season INTEGER,
            game_date TEXT,
            home_team TEXT,
            away_team TEXT,
            home_score INTEGER,
            away_score INTEGER,
            home_win INTEGER
        )
    """)

    conn.commit()
    conn.close()
    print("tables created")


create_tables()

tables created


## 3. Helper Functions

In [57]:
# Use BeautifulSoup to fetch and parse pages
def get_soup(url):
    try:
        time.sleep(DELAY)
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return BeautifulSoup(response.text, "html.parser")
    except Exception as e:
        print(f"fetch failed for {url} -- {e}")
        return None


# Helper functions for parsing tables using BeautifulSoup
def find_table(soup, table_id):
    # Try normal search first
    table = soup.find("table", {"id": table_id})
    if table is not None:
        return table

    # Search inside HTML comments
    # Comments are not parsed as part of the normal DOM, so we have to look through them manually
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment_text = str(comment)
        if table_id not in comment_text:
            continue
        comment_soup = BeautifulSoup(comment_text, "html.parser")
        table = comment_soup.find("table", {"id": table_id})
        if table is not None:
            return table

    return None

# Converts string -> int or None
def safe_int(value):
    if value is None:
        return None
    value = str(value).strip().replace(",", "")
    if value == "" or value == "-":
        return None
    try:
        return int(value)
    except ValueError:
        return None

# Converts string -> float or None
def safe_float(value):
    if value is None:
        return None
    value = str(value).strip().replace(",", "")
    if value == "" or value == "-":
        return None
    try:
        return float(value)
    except ValueError:
        return None


# Extracts text content of a table cell
def get_cell_text(row, stat_name):
    cell = row.find("td", {"data-stat": stat_name})
    if cell is None:
        cell = row.find("th", {"data-stat": stat_name})
    if cell is None:
        return ""
    return cell.get_text(strip=True)


# Extracts links from a table cell
def get_cell_link(row, stat_name):
    cell = row.find("td", {"data-stat": stat_name})
    if cell is None:
        cell = row.find("th", {"data-stat": stat_name})
    if cell is None:
        return None
    link = cell.find("a")
    if link is None:
        return None
    return link.get("href", None)

# Prints row to log file
# Prevents cell outputs from being too long
def log_row(row):
    with open("scrape_log.txt", "a") as f:
        f.write(str(row) + "\n")


print("Helpers ready")

Helpers ready


## 4. Scrape Teams from Season Pages

For each season, load the season summary page and extract the team names,
IDs, conferences, and divisions from the standings tables.

In [58]:
# Scrape teams for a given season
def scrape_teams_for_season(season):
    url = f"{BASE_URL}/leagues/NBA_{season}.html"
    season_label = f"{season - 1}-{str(season)[2:]}"
    print(f"[teams]: {season_label}")
    log_row(f"scraping teams for season: {season_label}")

    soup = get_soup(url)
    if soup is None:
        raise RuntimeError(f"Failed to fetch {url}")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    team_ids = []

    # Basketball Reference has Eastern and Western conference standings.
    # Table IDs vary by year; try multiple possibilities.
    conference_tables = [
        ("E", "Eastern"),
        ("W", "Western"),
    ]

    for conf_code, conf_name in conference_tables:
        table = find_table(soup, f"divs_standings_{conf_code}")
        if table is None:
            table = find_table(soup, f"confs_standings_{conf_code}")

        if table is None:
            print(f"(missing {conf_name} standings) ", end="")
            continue

        # Division headers are interspersed in the table, 
        # so track the current division as we go
        current_division = ""
        all_rows = table.find_all("tr")

        for row in all_rows:
            header_th = row.find("th", {"colspan": True})
            if header_th:
                current_division = header_th.get_text(strip=True)
                continue

            team_cell = row.find("th", {"data-stat": "team_name"})
            if team_cell is None:
                team_cell = row.find("td", {"data-stat": "team_name"})
            if team_cell is None:
                continue

            link = team_cell.find("a")
            if link is None:
                continue

            href = link.get("href", "")
            team_name = link.get_text(strip=True)

            # Parse team_id from URL like /teams/LAL/2020.html
            parts = href.strip("/").split("/")
            if len(parts) < 2:
                continue
            team_id = parts[1]

            team_ids.append(team_id)

            cursor.execute(
                "INSERT OR IGNORE INTO teams (team_id, team_name, conference, division) "
                "VALUES (?, ?, ?, ?)",
                (team_id, team_name, conf_name, current_division),
            )

    conn.commit()
    conn.close()

    print(f"found {len(team_ids)} teams")
    return team_ids

## 5. Scrape Team Season Stats

For each team-season, load the team page and extract:
- Win/loss record from the page meta section
- Advanced stats (Pace, ORtg, DRtg, SRS) from the team_misc table
- Roster player links (to build the set of player IDs to scrape later)

In [59]:
# Scrape a team's season page for team stats and roster player IDs.
def scrape_team_season(team_id, season, all_player_ids):
    url = f"{BASE_URL}/teams/{team_id}/{season}.html"
    print(f"{team_id} {season} ...", end=" ")
    log_row(f"scraping team season: {team_id} {season}")

    soup = get_soup(url)
    if soup is None:
        raise RuntimeError(f"Failed to fetch {url}")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    wins = None
    losses = None

    meta_div = soup.find("div", {"id": "meta"})
    if meta_div:
        meta_paragraphs = meta_div.find_all("p")
        for paragraph in meta_paragraphs:
            paragraph_text = paragraph.get_text()
            # Look for a line containing the record
            has_record_keyword = ("Record" in paragraph_text or "record" in paragraph_text)
            record_match = re.search(r'(\d+)-(\d+)', paragraph_text)
            if record_match and has_record_keyword:
                wins = safe_int(record_match.group(1))
                losses = safe_int(record_match.group(2))
                break

    # Compute win percentage
    win_pct = None
    if wins is not None and losses is not None:
        total_games = wins + losses
        if total_games > 0:
            win_pct = round(wins / total_games, 3)

    # Extract advanced team stats from team_misc table
    pace = None
    offensive_rating = None
    defensive_rating = None
    srs = None

    misc_table = find_table(soup, "team_misc")
    if misc_table:
        # The table has thead (column headers) and tbody (data rows).
        # We only want tbody rows -- otherwise we match the header text
        # like "Pace" instead of the actual value like "89.7".
        tbody = misc_table.find("tbody")
        if tbody:
            data_rows = tbody.find_all("tr")
        else:
            data_rows = misc_table.find_all("tr")

        for row in data_rows:
            pace_text = get_cell_text(row, "pace")
            if pace_text == "":
                continue
            pace_val = safe_float(pace_text)
            if pace_val is None:
                # this is a non-numeric row (like "Lg Rank"), skip it
                continue
            pace = pace_val
            srs = safe_float(get_cell_text(row, "srs"))
            offensive_rating = safe_float(get_cell_text(row, "off_rtg"))
            defensive_rating = safe_float(get_cell_text(row, "def_rtg"))
            break

    cursor.execute(
        "INSERT OR REPLACE INTO team_season_stats "
        "(team_id, season, wins, losses, win_pct, pace, "
        "offensive_rating, defensive_rating, srs) "
        "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (team_id, season, wins, losses, win_pct, pace,
         offensive_rating, defensive_rating, srs),
    )

    # Extract roster player IDs
    roster_table = find_table(soup, "roster")
    roster_count = 0
    if roster_table:
        roster_rows = roster_table.find_all("tr")
        for row in roster_rows:
            links = row.find_all("a")
            for link in links:
                href = link.get("href", "")
                if "/players/" in href and href.endswith(".html"):
                    parts = href.strip("/").split("/")
                    if len(parts) >= 3:
                        player_id = parts[2].replace(".html", "")
                        all_player_ids.add(player_id)
                        roster_count += 1

    conn.commit()
    conn.close()
    log_row(f"team season: {team_id} {season} -- wins: {wins}, losses: {losses}, win%: {win_pct}, pace: {pace}, off_rtg: {offensive_rating}, def_rtg: {defensive_rating}, srs: {srs}, num_players: {roster_count}")

## 6. Scrape Player Pages (Bio + Season Stats)

For each player, load their page once and extract:
- Biographical info (name, height, weight, position, draft) from the meta section
- Per-game stats from the `per_game` table
- Advanced stats (PER, TS%, WS, BPM, VORP) from the `advanced` table

The two stat tables are merged by (season, team_id) key before inserting.

In [60]:
# Scrape a player's Basketball Reference page.
def scrape_player_page(player_id):
    # Player pages are organized by the first letter of the player_id
    first_letter = player_id[0]
    url = f"{BASE_URL}/players/{first_letter}/{player_id}.html"

    soup = get_soup(url)
    if soup is None:
        raise RuntimeError(f"Failed to fetch {url}")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    ## Player bio and draft info

    full_name = ""
    birth_date = None
    height = None
    weight = None
    position = None
    shoots = None
    draft_year = None
    draft_round = None
    draft_pick = None
    draft_team = None

    # Name
    heading = soup.find("h1")
    if heading:
        name_span = heading.find("span")
        if name_span:
            full_name = name_span.get_text(strip=True)
        else:
            full_name = heading.get_text(strip=True)

    # Meta section fields
    meta_div = soup.find("div", {"id": "meta"})
    if meta_div:
        paragraphs = meta_div.find_all("p")

        for paragraph in paragraphs:
            text = paragraph.get_text(strip=True)

            # Position
            if "Position:" in text:
                pos_match = re.search(
                    r'Position:\s*(.+?)(?:\s*\u25aa|\s*Shoots:|$)', text
                )
                if pos_match:
                    position = pos_match.group(1).strip()

            # Shooting hand
            if "Shoots:" in text:
                shoots_match = re.search(r'Shoots:\s*(\w+)', text)
                if shoots_match:
                    shoots = shoots_match.group(1).strip()

            # Height / weight
            height_match = re.search(r'(\d+-\d+)', text)
            weight_match = re.search(r'(\d+)lb', text)
            if height_match and ("lb" in text or "cm" in text):
                height = height_match.group(1)
            if weight_match:
                weight = safe_int(weight_match.group(1))

            # Draft info
            if "Draft:" in text:
                draft_team_match = re.search(r'Draft:\s*(.+?),', text)
                if draft_team_match:
                    draft_team = draft_team_match.group(1).strip()

                round_match = re.search(r'(\d+)\w*\s*round', text)
                if round_match:
                    draft_round = safe_int(round_match.group(1))

                pick_match = re.search(r'(\d+)\w*\s*pick', text)
                if pick_match:
                    draft_pick = safe_int(pick_match.group(1))

                year_match = re.search(r'(\d{4})\s*NBA\s*Draft', text)
                if year_match:
                    draft_year = safe_int(year_match.group(1))

        # Birth date
        birth_span = meta_div.find("span", {"id": "necro-birth"})
        if birth_span:
            birth_date = birth_span.get("data-birth", None)
            if birth_date is None:
                birth_date = birth_span.get_text(strip=True)

    cursor.execute(
        "INSERT OR REPLACE INTO players "
        "(player_id, full_name, birth_date, height, weight, position, shoots, "
        "draft_year, draft_round, draft_pick, draft_team) "
        "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (player_id, full_name, birth_date, height, weight, position, shoots,
         draft_year, draft_round, draft_pick, draft_team),
    )

    ## Player season stats
    
    ### Per-game season stats
    per_game_data = {}  # (season_year, team_id): {games, games_started, minutes, points, rebounds, assists}

    # Basketball Reference uses table id "per_game_stats" (not "per_game")
    per_game_table = find_table(soup, "per_game_stats")
    if per_game_table:
        tbody = per_game_table.find("tbody")
        rows = tbody.find_all("tr") if tbody else per_game_table.find_all("tr")

        for row in rows:
            # Skip sub-header rows
            row_classes = row.get("class", [])
            if "thead" in row_classes or "partial_table" in row_classes:
                continue

            # Parse season text like "2019-20" -> season_year = 2020
            # Basketball Reference uses data-stat="year_id" for the season column
            season_text = get_cell_text(row, "year_id")
            if season_text == "":
                continue
            season_match = re.match(r'(\d{4})-(\d{2})', season_text)
            if season_match is None:
                continue
            season_year = int(season_match.group(1)) + 1

            if season_year < 2000 or season_year > 2025:
                continue

            # Get team -- data-stat is "team_name_abbr" on player pages
            team_text = get_cell_text(row, "team_name_abbr")
            if team_text == "TOT":
                continue

            team_link = get_cell_link(row, "team_name_abbr")
            if team_link:
                link_parts = team_link.strip("/").split("/")
                team_id_val = link_parts[1] if len(link_parts) >= 2 else team_text
            else:
                team_id_val = team_text

            if team_id_val == "":
                continue

            # data-stat names: "games", "games_started" (not "g", "gs")
            games = safe_int(get_cell_text(row, "games"))
            games_started = safe_int(get_cell_text(row, "games_started"))
            minutes = safe_float(get_cell_text(row, "mp_per_g"))
            points = safe_float(get_cell_text(row, "pts_per_g"))
            rebounds = safe_float(get_cell_text(row, "trb_per_g"))
            assists = safe_float(get_cell_text(row, "ast_per_g"))

            key = (season_year, team_id_val)
            per_game_data[key] = {
                "games": games,
                "games_started": games_started,
                "minutes": minutes,
                "points": points,
                "rebounds": rebounds,
                "assists": assists,
            }

    ### Advanced season stats
    advanced_data = {}  # (season_year, team_id): {per, ts_pct, ws, bpm, vorp}

    advanced_table = find_table(soup, "advanced")
    if advanced_table:
        tbody = advanced_table.find("tbody")
        rows = tbody.find_all("tr") if tbody else advanced_table.find_all("tr")

        for row in rows:
            row_classes = row.get("class", [])
            if "thead" in row_classes or "partial_table" in row_classes:
                continue

            season_text = get_cell_text(row, "year_id")
            if season_text == "":
                continue
            season_match = re.match(r'(\d{4})-(\d{2})', season_text)
            if season_match is None:
                continue
            season_year = int(season_match.group(1)) + 1

            if season_year < 2000 or season_year > 2025:
                continue

            team_text = get_cell_text(row, "team_name_abbr")
            if team_text == "TOT":
                continue

            team_link = get_cell_link(row, "team_name_abbr")
            if team_link:
                link_parts = team_link.strip("/").split("/")
                team_id_val = link_parts[1] if len(link_parts) >= 2 else team_text
            else:
                team_id_val = team_text

            per_val = safe_float(get_cell_text(row, "per"))
            ts_pct = safe_float(get_cell_text(row, "ts_pct"))
            ws = safe_float(get_cell_text(row, "ws"))
            bpm = safe_float(get_cell_text(row, "bpm"))
            vorp = safe_float(get_cell_text(row, "vorp"))

            key = (season_year, team_id_val)
            advanced_data[key] = {
                "per": per_val,
                "ts_pct": ts_pct,
                "ws": ws,
                "bpm": bpm,
                "vorp": vorp,
            }

    ### Merge per-game and advanced, then insert
    all_keys = set(per_game_data.keys()) | set(advanced_data.keys())

    for key in all_keys:
        season_year, team_id_val = key

        pg = per_game_data.get(key, {})
        adv = advanced_data.get(key, {})

        cursor.execute(
            "INSERT OR REPLACE INTO player_season_stats "
            "(player_id, season, team_id, games, games_started, minutes, "
            "points, rebounds, assists, per, ts_pct, ws, bpm, vorp) "
            "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (
                player_id,
                season_year,
                team_id_val,
                pg.get("games"),
                pg.get("games_started"),
                pg.get("minutes"),
                pg.get("points"),
                pg.get("rebounds"),
                pg.get("assists"),
                adv.get("per"),
                adv.get("ts_pct"),
                adv.get("ws"),
                adv.get("bpm"),
                adv.get("vorp"),
            ),
        )

    conn.commit()
    conn.close()
    log_row(f"player: {full_name} -- {len(all_keys)} season entries inserted")

## 7. Scrape Games

For each season, load the schedule page and follow the monthly sub-links.
Each month page contains a schedule table with game results.

In [61]:
# Scrape all games for a season, including date, teams, scores, and winner.
def scrape_games_for_season(season):
    # Games are organized by season on a main schedule page, 
    # which links to monthly sub-pages for some seasons.
    url = f"{BASE_URL}/leagues/NBA_{season}_games.html"
    season_label = f"{season - 1}-{str(season)[2:]}"
    print(f"[games] {season_label} ...", end=" ")
    log_row(f"scraping games for season: {season_label}")

    soup = get_soup(url)
    if soup is None:
        raise RuntimeError(f"Failed to fetch {url}")

    # Collect monthly schedule page links
    month_links = []
    filter_div = soup.find("div", {"class": "filter"})
    if filter_div:
        links = filter_div.find_all("a")
        for link in links:
            href = link.get("href", "")
            if "games" in href:
                full_url = BASE_URL + href
                month_links.append(full_url)

    # If no month links found, the current page may have the schedule directly
    if len(month_links) == 0:
        month_links = [url]

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    games_inserted = 0

    for month_url in month_links:
        # Avoid re-fetching the page we already have
        if month_url == url:
            month_soup = soup
        else:
            month_soup = get_soup(month_url)

        if month_soup is None:
            raise RuntimeError(f"Failed to fetch month page {month_url}")

        schedule_table = find_table(month_soup, "schedule")
        if schedule_table is None:
            continue

        tbody = schedule_table.find("tbody")
        rows = tbody.find_all("tr") if tbody else schedule_table.find_all("tr")

        for row in rows:
            # Skip header rows
            row_classes = row.get("class", [])
            if "thead" in row_classes:
                continue

            # Box score link gives us the game_id
            box_link = get_cell_link(row, "box_score_text")
            if box_link is None:
                continue

            # Parse game_id from box link (i.e. /boxscores/202001010LAL.html)
            game_id_with_ext = box_link.split("/")[-1]
            game_id = game_id_with_ext.replace(".html", "")

            # Date
            date_cell = row.find("th", {"data-stat": "date_game"})
            if date_cell is None:
                continue
            game_date = date_cell.get("csk", date_cell.get_text(strip=True))

            # Away team
            away_link = get_cell_link(row, "visitor_team_name")
            away_team_id = ""
            if away_link:
                parts = away_link.strip("/").split("/")
                if len(parts) >= 2:
                    away_team_id = parts[1]

            # Home team
            home_link = get_cell_link(row, "home_team_name")
            home_team_id = ""
            if home_link:
                parts = home_link.strip("/").split("/")
                if len(parts) >= 2:
                    home_team_id = parts[1]

            # Scores
            away_score = safe_int(get_cell_text(row, "visitor_pts"))
            home_score = safe_int(get_cell_text(row, "home_pts"))

            # Skip rows with missing scores (e.g., future games)
            if home_score is None or away_score is None:
                continue

            home_win = 1 if home_score > away_score else 0

            cursor.execute(
                "INSERT OR REPLACE INTO games "
                "(game_id, season, game_date, home_team, away_team, "
                "home_score, away_score, home_win) "
                "VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                (game_id, season, game_date, home_team_id, away_team_id,
                 home_score, away_score, home_win),
            )
            games_inserted += 1

    conn.commit()
    conn.close()
    print(f"{games_inserted} games inserted")
    log_row(f"total games inserted for season {season_label}: {games_inserted}")

## 8. Main Orchestration

Run the full scraping pipeline:
1. Create tables
2. For each season: scrape teams, then scrape each team's season page (collecting player IDs)
3. Scrape all discovered player pages
4. Scrape game results for each season

Each step uses `INSERT OR REPLACE`, so the script can be safely re-run if interrupted.

In [62]:
MAX_RETRIES = 5


# Main function
# Execute the scraping steps with retries
def main():
    
    create_tables()
    
    permanently_failed = []
    all_player_ids = set()
    
    ## Scrape team season pages to get team IDs and rosters
    failed_seasons = {}  # season: attempt_count
    for season in SEASONS:
        failed_seasons[season] = 0

    seasons_to_scrape = list(SEASONS)
    team_ids_by_season = {}  # season: list of team_ids

    while len(seasons_to_scrape) > 0:
        still_failing = []
        for season in seasons_to_scrape:
            season_label = f"{season - 1}-{str(season)[2:]}"
            try:
                team_ids = scrape_teams_for_season(season)
                if len(team_ids) == 0:
                    raise RuntimeError("No teams found on page")
                team_ids_by_season[season] = team_ids
            except Exception as e:
                failed_seasons[season] += 1
                if failed_seasons[season] >= MAX_RETRIES:
                    msg = f"season page {season_label}: {e}"
                    print(f"GAVE UP on {msg}")
                    permanently_failed.append(msg)
                    team_ids_by_season[season] = []
                else:
                    print(f"Will retry season {season_label} (attempt {failed_seasons[season]}/{MAX_RETRIES})")
                    still_failing.append(season)
        seasons_to_scrape = still_failing

    # Init scrape team season pages
    print()
    print(f"Init scraping team season pages for all teams and seasons")
    failed_team_seasons = {}  # (team_id, season): attempt_count
    team_seasons_to_scrape = []

    for season in SEASONS:
        for team_id in team_ids_by_season.get(season, []):
            team_seasons_to_scrape.append((team_id, season))
            failed_team_seasons[(team_id, season)] = 0

    while len(team_seasons_to_scrape) > 0:
        still_failing = []
        for team_id, season in team_seasons_to_scrape:
            try:
                scrape_team_season(team_id, season, all_player_ids)
            except Exception as e:
                failed_team_seasons[(team_id, season)] += 1
                attempts = failed_team_seasons[(team_id, season)]
                if attempts >= MAX_RETRIES:
                    msg = f"team {team_id} {season}: {e}"
                    log_row(f"FAILED team season after {MAX_RETRIES} attempts: {team_id} {season} -- error: {e}")
                    permanently_failed.append(msg)
                else:
                    log_row(f"failed team season: {team_id} {season} -- attempt {attempts} -- error: {e}")
                    still_failing.append((team_id, season))
        team_seasons_to_scrape = still_failing

    print()
    print(f"total unique player IDs collected from team rosters: {len(all_player_ids)}")
    log_row(f"total unique player IDs collected: {len(all_player_ids)}")

    # nit scrape player pages to get player bios and season stats
    print()
    print("Init scraping player pages ")

    player_list = sorted(all_player_ids)
    failed_players = {}  # player_id: attempt_count

    for pid in player_list:
        failed_players[pid] = 0

    players_to_scrape = list(player_list)

    while len(players_to_scrape) > 0:
        still_failing = []
        for pid in players_to_scrape:
            try:
                attempt_num = failed_players[pid] + 1
                log_row(f"scraping player page: {pid} -- attempt {attempt_num}")
                scrape_player_page(pid)
            except Exception as e:
                failed_players[pid] += 1
                attempts = failed_players[pid]
                if attempts >= MAX_RETRIES:
                    msg = f"player {pid}: {e}"
                    log_row(f"FAILED player after {MAX_RETRIES} attempts: {pid} -- error: {e}")
                    permanently_failed.append(msg)
                else:
                    log_row(f"failed player: {pid} -- attempt {attempts} -- error: {e}")
                    still_failing.append(pid)
        players_to_scrape = still_failing
        if len(still_failing) > 0:
            log_row(f"retrying {len(still_failing)} failed players: {still_failing}")

    # Init scrape game results for each season
    print()
    print("Init scraping game results for each season")

    failed_game_seasons = {}  # season: attempt_count
    for season in SEASONS:
        failed_game_seasons[season] = 0

    game_seasons_to_scrape = list(SEASONS)

    while len(game_seasons_to_scrape) > 0:
        still_failing = []
        for season in game_seasons_to_scrape:
            try:
                scrape_games_for_season(season)
            except Exception as e:
                failed_game_seasons[season] += 1
                attempts = failed_game_seasons[season]
                season_label = f"{season - 1}-{str(season)[2:]}"
                if attempts >= MAX_RETRIES:
                    msg = f"games {season_label}: {e}"
                    log_row(f"FAILED games for season after {MAX_RETRIES} attempts: {season_label} -- error: {e}")
                    permanently_failed.append(msg)
                else:
                    log_row(f"failed games for season {season_label} -- attempt {attempts} -- error: {e}")
                    still_failing.append(season)
        game_seasons_to_scrape = still_failing

    print()
    print("SCRAPING COMPLETE")

    if len(permanently_failed) == 0:
        log_row("everything succeeded, no failures")
    else:
        log_row(f"{len(permanently_failed)} item(s) failed all {MAX_RETRIES} attempts:")
        for item in permanently_failed:
            log_row(f"  * {item}")

main()

tables created
[teams]: 1999-00
found 29 teams
[teams]: 2000-01
found 29 teams
[teams]: 2001-02
found 29 teams
[teams]: 2002-03
found 29 teams
[teams]: 2003-04
found 29 teams
[teams]: 2004-05
found 30 teams
[teams]: 2005-06
found 30 teams
[teams]: 2006-07
found 30 teams
[teams]: 2007-08
found 30 teams
[teams]: 2008-09
found 30 teams
[teams]: 2009-10
found 30 teams
[teams]: 2010-11
found 30 teams
[teams]: 2011-12
found 30 teams
[teams]: 2012-13
found 30 teams
[teams]: 2013-14
found 30 teams
[teams]: 2014-15
found 30 teams
[teams]: 2015-16
found 30 teams
[teams]: 2016-17
found 30 teams
[teams]: 2017-18
found 30 teams
[teams]: 2018-19
found 30 teams
[teams]: 2019-20
found 30 teams
[teams]: 2020-21
found 30 teams
[teams]: 2021-22
found 30 teams
[teams]: 2022-23
found 30 teams
[teams]: 2023-24
found 30 teams
[teams]: 2024-25
found 30 teams

Init scraping team season pages for all teams and seasons
MIA 2000 ... NYK 2000 ... PHI 2000 ... ORL 2000 ... BOS 2000 ... NJN 2000 ... WAS 2000 ... IND

## 9. Validation

Run queries to verify the database was populated correctly.

In [ ]:
# Run a few validation queries against the database to spot check results
def validate_database():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # row counts
    print("row counts:")
    tables = ["players", "teams", "player_season_stats", "team_season_stats", "games"]
    for table_name in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"  {table_name}: {count:,}")

    # spot check lebron
    print()
    print("spot check -- LeBron James (jamesle01):")
    cursor.execute("SELECT * FROM players WHERE player_id = 'jamesle01'")
    row = cursor.fetchone()
    if row:
        print(f"  name: {row[1]}")
        print(f"  born: {row[2]}")
        print(f"  height/weight: {row[3]}, {row[4]} lb")
        print(f"  position: {row[5]}, shoots: {row[6]}")
        print(f"  draft: {row[10]} / round {row[8]} / pick {row[9]} / {row[7]}")
    else:
        print("  not found in db")

    cursor.execute(
        "SELECT season, team_id, points, rebounds, assists"
        "FROM player_season_stats "
        "WHERE player_id = 'jamesle01' "
        "ORDER BY season"
    )
    lebron_seasons = cursor.fetchall()
    print(f"  seasons on file: {len(lebron_seasons)}")
    for s in lebron_seasons:
        print(f"    {s[0]} ({s[1]}): {s[2]} ppg / {s[3]} rpg / {s[4]} apg")

    # spot check lakers 2020
    print()
    print("spot check -- LAL 2019-20:")
    cursor.execute(
        "SELECT * FROM team_season_stats WHERE team_id = 'LAL' AND season = 2020"
    )
    row = cursor.fetchone()
    if row:
        print(f"  record: {row[2]}-{row[3]} (win%: {row[4]})")
        print(f"  pace: {row[5]}, ortg: {row[6]}, drtg: {row[7]}, srs: {row[8]}")
    else:
        print("  not found in db")

    # games check
    print()
    print("spot check -- games:")
    cursor.execute("SELECT COUNT(*) FROM games WHERE season = 2024")
    count = cursor.fetchone()[0]
    print(f"  2023-24 season: {count} games")

    cursor.execute("SELECT * FROM games ORDER BY game_date LIMIT 5")
    sample_games = cursor.fetchall()
    print(f"  first 5 games in db:")
    for game in sample_games:
        winner = "home W" if game[7] else "away W"
        print(f"    {game[2]} -- {game[4]} at {game[3]}, {game[6]}-{game[5]} ({winner})")

    # season coverage
    print()
    cursor.execute("SELECT DISTINCT season FROM team_season_stats ORDER BY season")
    seasons_found = [r[0] for r in cursor.fetchall()]
    if seasons_found:
        print(f"seasons in team_season_stats: {seasons_found[0]} to {seasons_found[-1]} ({len(seasons_found)} total)")
    else:
        print("no seasons found in team_season_stats")

    conn.close()
    print()
    print("done validating")


validate_database()

row counts:
  players: 2,551
  teams: 37
  player_season_stats: 12,667
  team_season_stats: 775
  games: 33,392

spot check -- LeBron James (jamesle01):
  name: LeBron James
  born: 1984-12-30
  height/weight: 6-9, 250 lb
  position: Small Forward, Power Forward, Point Guard, Center, and Shooting Guard, shoots: Right
  draft: Cleveland Cavaliers / round 1 / pick 1 / 2003


OperationalError: no such column: season